In [1]:
import numpy as np
import pandas as pd
import re
import string
import pickle

In [2]:
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, '')
    return text

In [3]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [4]:
# with open('../static/model/new_model.pickle', 'rb') as f:
#    new_model = pickle.load(f)

In [5]:
with open('../static/model/corpora/stopwords/sinhala', 'r', encoding='utf-8') as file:
    sw = file.read().splitlines()

In [6]:
vocab = pd.read_csv('../static/model/vocabulary.txt', header=None)
tokens = vocab[0].tolist()

In [7]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [8]:
def preprocessing(text):
    data = pd.DataFrame([text], columns=['Title'])
    data["Title"] = data["Title"].apply(lambda x: " ".join(x.lower() for x in x.split()))
    data["Title"] = data['Title'].apply(lambda x: " ".join(re.sub(r'^https?:\/\/.*[\r\n]*', '', x, flags=re.MULTILINE) for x in x.split()))
    data["Title"] = data["Title"].apply(remove_punctuations)
    data["Title"] = data['Title'].str.replace('\d+', '', regex=True)
    data["Title"] = data["Title"].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
    data["Title"] = data["Title"].apply(lambda x: " ".join(ps.stem(x) for x in x.split()))
    return data["Title"]

In [9]:
def vectorizer(ds, vocabulary):
    vectorized_lst = []
    
    for sentence in ds:
        sentence_lst = np.zeros(len(vocabulary))
        
        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_lst[i] = 1
                
        vectorized_lst.append(sentence_lst)
        
    vectorized_lst_new = np.asarray(vectorized_lst, dtype=np.float32)
    
    return vectorized_lst_new

In [10]:
def get_prediction(vectorized_text):
    prediction = model.predict(vectorized_text)
    # prediction = new_model.predict(vectorized_text)
    if prediction == 0:
        return 'International'
    elif prediction == 1:
        return 'Sport'
    elif prediction == 2:
        return 'Business'

In [11]:
# headline1 = "ශ්‍රී ලංකා අයර්ලන්ත ක්‍රිකට් තරගය අද"
headline1 = "ආසියානු ක්‍රිකට් කුසලානයේ තරග 9ක් ලංකාවේදී"

preprocessed_txt = preprocessing(headline1)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'Sport'

In [12]:
# headline2 = "පාපන්දු සහ රග්බි තහනමෙන් මුදාගන්න ඉදිරි සති දෙකේ සාකච්ඡා"
headline2 = "ජාතික වොලිබෝල් කාන්තා කණ්ඩායම නේපාලයට"
preprocessed_txt = preprocessing(headline2)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'Sport'

In [13]:
# headline3 = "මොස්කව් ඇල්ලීම නවතී - වැග්නර් හමුදාව පසුබසී"
headline3 = "ගැටුම් මැද පුටින්ගෙන් ප්‍රකාශයක්"
preprocessed_txt = preprocessing(headline3)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'International'

In [14]:
# headline4 = "ලොවක් කතා කරන ටයිටෑන් සබ්මැරීනයේ ආරක්ෂාව ගැන ප්‍රශ්නයක් "
headline4 = "රුසියානු ඇන්ටනෝව් යානාව පැහැරගත් කැනඩාව එය යුක්රේනයට භාර දෙයි"
preprocessed_txt = preprocessing(headline4)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'International'

In [15]:
headline5 = "මහජන බැංකුවට රුපියල් බිලියන 4.6ක ඒකාබද්ධ බදු පසු ලාභයක් "
# headline5 = "මහජන බැංකුවේ ‘වාසි කෝටියයි’ සුපිරි ජයග්‍රාහකයාට කෝටියක් පිරිනැමේ"
preprocessed_txt = preprocessing(headline4)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'International'

In [16]:
headline6 = "අපනයන-නැඹුරු නිෂ්පාදන ආර්ථිකය කරා යන කර්මාන්ත 2023 ජාතික කර්මාන්ත ප්‍රදර්ශනය"
# headline6 = "SLT සමූහයේ ප්‍රථම කාර්තුවේ ස්ථාවර ආදායමක් වාර්තා කරයි"
preprocessed_txt = preprocessing(headline4)
vectorized_txt = vectorizer(preprocessed_txt, tokens)
prediction = get_prediction(vectorized_txt)
prediction

'International'